# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# Import necessary libraries for data manipulation and database interaction
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


# Load the messages dataset
try:
    messages = pd.read_csv('messages.csv')
    print("Messages dataset loaded successfully.")
    display(messages.head(7))  # Display the first 7 rows for a quick look
except FileNotFoundError:
    print("Error: 'messages.csv' file not found.")
except pd.errors.EmptyDataError:
    print("Error: 'messages.csv' is empty.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Messages dataset loaded successfully.


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct


In [2]:
messages['message'][0]

'Weather update - a cold front from Cuba that could pass over Haiti'

###  Inspect Data and Check for Missing Values
- **Objective**: Get a quick overview of the `messages` and `categories` datasets to understand their structure and check for missing values.
- **Actions**: Used `info()` to summarize the datasets and `isnull().sum()` to identify missing values.

### Clean and Split Categories Dataset
- **Objective**: Transform the `categories` column into individual category columns with binary (0/1) values.
- **Actions**:
  - Split the `categories` column at each `;` and create new columns.
  - Extract and rename columns based on the first part of the category string.
  - Convert the category values (e.g., `related-1`) into numeric (0 or 1).

###  Merge Datasets
- **Objective**: Combine the `messages` and cleaned `categories` datasets based on their common `id` column.
- **Actions**: Performed a merge operation and inspected the shape and content of the merged dataset.

### Handle Missing Data
- **Objective**: Ensure that the merged dataset has no missing values.
- **Actions**: Checked for missing data in the merged dataset and dropped any rows containing missing values.


In [3]:
# Load the categories dataset
try:
    categories = pd.read_csv('categories.csv')
    print("Categories dataset loaded successfully.")
    display(categories.head(7))  # Display the first 7 rows for a quick look
except FileNotFoundError:
    print("Error: 'categories.csv' file not found.")
except pd.errors.EmptyDataError:
    print("Error: 'categories.csv' is empty.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Categories dataset loaded successfully.


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...
5,14,related-0;request-0;offer-0;aid_related-0;medi...
6,15,related-1;request-0;offer-0;aid_related-0;medi...


In [4]:
# Inspect the messages dataset
print("Messages dataset overview:")
print(messages.info())  # Provides a concise summary of the dataset
print("\nMissing values in messages:")
print(messages.isnull().sum())  # Check for missing values in the messages dataset

# Inspect the categories dataset
print("\nCategories dataset overview:")
print(categories.info())  # Provides a concise summary of the dataset
print("\nMissing values in categories:")
print(categories.isnull().sum())  # Check for missing values in the categories dataset

Messages dataset overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        26248 non-null  int64 
 1   message   26248 non-null  object
 2   original  10184 non-null  object
 3   genre     26248 non-null  object
dtypes: int64(1), object(3)
memory usage: 820.4+ KB
None

Missing values in messages:
id              0
message         0
original    16064
genre           0
dtype: int64

Categories dataset overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          26248 non-null  int64 
 1   categories  26248 non-null  object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB
None

Missing values in categories:
id            0
categories    0
dtype: int64


In [4]:
# Ensure the 'id' column is retained
categories_clean = categories.copy()  # Create a copy to preserve the original structure

# Split the categories into separate columns
categories_split = categories_clean['categories'].str.split(';', expand=True)

# Extract the first row to get the new column names
row = categories_split.iloc[0]
category_colnames = row.apply(lambda x: x.split('-')[0])  # Extract column names
categories_split.columns = category_colnames  # Rename columns

# Convert category values to just 0 or 1
for column in categories_split:
    categories_split[column] = categories_split[column].str[-1].astype(int)

# Add the 'id' column back to categories_split
categories_split['id'] = categories_clean['id']



In [5]:
# Merge messages and categories datasets on 'id'
df = messages.merge(categories_split, left_on='id', right_on='id')

# Check the shape and preview the merged dataframe
print(f"\nMerged dataset shape: {df.shape}")
display(df.head())


Merged dataset shape: (26386, 40)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [7]:
df.earthquake.value_counts

<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
26381    0
26382    0
26383    0
26384    0
26385    0
Name: earthquake, Length: 26386, dtype: int32>

In [9]:
# Merge messages and categories datasets on 'id'
df = messages.merge(categories_split, left_on='id', right_on='id')

# Check the shape and preview the merged dataframe
print(f"\nMerged dataset shape: {df.shape}")
display(df.head())


Merged dataset shape: (26386, 40)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Check for missing values in the merged dataset
print("\nMissing values in merged dataset:")
print(df.isnull().sum())

# Drop rows with missing values
df_cleaned = df.dropna()

print(f"\nCleaned dataset shape: {df_cleaned.shape}")
display(df_cleaned.head())


Missing values in merged dataset:
id                            0
message                       0
original                  16140
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_c

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Remove Duplicates
- **Objective**: Check for and remove any duplicate rows in the dataset.
- **Actions**:
  - Used `duplicated()` to identify duplicates.
  - Dropped duplicates using `drop_duplicates()` if any were found.

---



In [11]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Remove duplicates if any are found
if duplicate_count > 0:
    df_cleaned = df.drop_duplicates()
    print(f"Dataset shape after removing duplicates: {df_cleaned.shape}")
else:
    df_cleaned = df


Number of duplicate rows: 170
Dataset shape after removing duplicates: (26216, 40)


### Save the cleaned dataset to an SQLite database
You can achieve this by using pandas' [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) in combination with the SQLAlchemy library. Make sure to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it in the code below.


In [15]:
engine = create_engine('sqlite:///Messages.db')
df.to_sql('Messages', engine, index=False, if_exists='replace')

26386

### Complete `etl_pipeline.py` using this notebook
Use the provided template in the Resources folder to write a script that performs the necessary steps to build a database based on user-specified datasets. Alternatively, you can finish the `etl_pipeline.py` script in the `Project Workspace IDE`, which will be available later in the classroom.


In [13]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
database_filepath = "disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df.to_sql(table_name, engine, index=False, if_exists='replace')

NameError: name 'os' is not defined